In [ ]:

pip install peft==v0.10.0


In [ ]:
pip install datasets

In [ ]:
pip install transformers

In [1]:
# Loading the model
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import AutoPeftModelForSequenceClassification

model_name = "gpt2"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Creating a PEFT model (no need to create a custom config)
peft_model = AutoPeftModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

# Evaluating the model
from datasets import load_dataset

dataset_name = "stanfordnlp/imdb"
imdb = load_dataset(dataset_name)

# Use a smaller subset of the dataset
small_train_dataset = imdb["train"].shuffle(seed=42).select(range(100))  # Further reduce to 100 samples for training
small_test_dataset = imdb["test"].shuffle(seed=42).select(range(20))    # Further reduce to 20 samples for testing

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")

tokenized_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = small_test_dataset.map(tokenize_function, batched=True)

# Convert datasets to PyTorch format
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Evaluate the model
import numpy as np
import torch
from torch.utils.data import DataLoader

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

def evaluate(model, dataset, batch_size=16):
    model.eval()
    eval_dataloader = DataLoader(dataset, batch_size=batch_size)
    all_logits = []
    all_labels = []

    for batch in eval_dataloader:
        inputs = {
            'input_ids': batch['input_ids'].squeeze(1),
            'attention_mask': batch['attention_mask'].squeeze(1)
        }
        labels = batch["label"].to(model.device)

        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        all_logits.append(logits.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

    all_logits = np.concatenate(all_logits, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    return compute_metrics((all_logits, all_labels))

# Evaluate the base model before fine-tuning
base_eval_results = evaluate(peft_model, tokenized_test_dataset)
print(f"Initial Evaluation: {base_eval_results}")

# Training the model
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Reduce batch size
    per_device_eval_batch_size=4,   # Reduce batch size
    num_train_epochs=1,             # Reduce the number of epochs
    weight_decay=0.01,
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# Saving the trained model
peft_model.save_pretrained("./peft_model")

# Loading the trained PEFT model
peft_model = AutoPeftModelForSequenceClassification.from_pretrained("./peft_model")

# Re-evaluate the PEFT model
peft_eval_results = evaluate(peft_model, tokenized_test_dataset)
print(f"PEFT Model Evaluation: {peft_eval_results}")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Can't find 'adapter_config.json' at 'gpt2'